In [ ]:
# Création des informations/résultats d'entrainement
df_train = pd.DataFrame(y_train, columns=['y_train'])
df_train['y_eval'] = classifier.predict(X_train)
df_train['y_eval_proba'] = classifier.predict_proba(X_train)[:, 1]  # Probabilité d'avoir un "événement" (arrêté)
df_train.index = dates_train
df_train.sort_index(inplace=True)

# Création des informations/résultats de test
df_pred = pd.DataFrame(y_test, columns=['y_test'])
df_pred['y_pred'] = classifier.predict(X_test)
df_pred['y_pred_proba'] = classifier.predict_proba(X_test)[:, 1]  # Probabilité d'avoir un "événement" (arrêté)
df_pred.index = dates_test
df_pred.sort_index(inplace=True)


# Graphique de visualisation des résultats sur la période d'entrainement
time = df_train.index
actual = df_train['y_train']
predicted = df_train['y_eval']
probabilities = df_train['y_eval_proba']

# Time Series of Actual vs. Predicted
fig = go.Figure()
# lines+markers
fig.add_trace(go.Scatter(x=time, y=actual, mode='markers', name='Actual', line=dict(width=2)))
fig.add_trace(go.Scatter(x=time, y=predicted, mode='markers', name='Predicted', marker=dict(symbol='circle-open-dot'), line=dict(width=1, dash='dot')))
fig.add_trace(go.Scatter(x=time, y=probabilities, mode='markers', name='Predicted Probability', marker=dict(symbol='diamond-open')))
fig.update_layout(title='Time Series of Observed vs. Evaluated',
                  xaxis_title='Time',
                  yaxis_title='Outcome',
                legend=dict(
                    x=0.5,
                    y=-0.3,
                    orientation='h',
                    xanchor='center',
                    yanchor='top'
    )
                 ,
                 height=500)
fig.show()

# Graphique de visualisation des résultats sur la période de test
time = df_pred.index
actual = df_pred['y_test']
predicted = df_pred['y_pred']
probabilities = df_pred['y_pred_proba']

# Time Series of Actual vs. Predicted
fig = go.Figure()
# lines+markers
fig.add_trace(go.Scatter(x=time, y=actual, mode='markers', name='Actual', line=dict(width=2)))
fig.add_trace(go.Scatter(x=time, y=predicted, mode='markers', name='Predicted', marker=dict(symbol='circle-open-dot'), line=dict(width=1, dash='dot')))
fig.add_trace(go.Scatter(x=time, y=probabilities, mode='markers', name='Predicted Probability', marker=dict(symbol='diamond-open')))
fig.update_layout(title='Time Series of Actual vs. Predicted',
                  xaxis_title='Time',
                  yaxis_title='Outcome',
                legend=dict(
                    x=0.5,
                    y=-0.3,
                    orientation='h',
                    xanchor='center',
                    yanchor='top'
    )
                 ,
                 height=500)
fig.show()


# Matrices de confusion sur les périodes d'entrainement et de test
cm_train = confusion_matrix(df_train['y_train'], df_train['y_eval'])
z_text_train = [[str(y) for y in x] for x in cm_train]  # Convert numbers to string for annotations
fig_cm_train = go.Figure(data=go.Heatmap(z=cm_train, x=['Predicted: No', 'Predicted: Yes'], 
                                         y=['Actual: No', 'Actual: Yes'], 
                                         colorscale='Blues', text=z_text_train, texttemplate="%{text}"))
fig_cm_train.update_layout(title='Confusion Matrix - Training', xaxis_title="Predicted Value", 
                           yaxis_title="Actual Value", width=600, height=600)
fig_cm_train.show()

# Confusion Matrix for Testing Data
cm_pred = confusion_matrix(df_pred['y_test'], df_pred['y_pred'])
z_text_pred = [[str(y) for y in x] for x in cm_pred]  # Convert numbers to string for annotations
fig_cm_pred = go.Figure(data=go.Heatmap(z=cm_pred, x=['Predicted: No', 'Predicted: Yes'], 
                                         y=['Actual: No', 'Actual: Yes'], 
                                         colorscale='Blues', text=z_text_pred, texttemplate="%{text}"))
fig_cm_pred.update_layout(title='Confusion Matrix - Testing', xaxis_title="Predicted Value", 
                           yaxis_title="Actual Value", width=600, height=600)
fig_cm_pred.show()



# Courbes ROC sur les périodes d'entrainement et de test

# Compute ROC curve and ROC area - training
fpr1, tpr1, thresholds1 = roc_curve(df_train['y_train'], df_train['y_eval_proba'])
roc_auc1 = roc_auc_score(df_train['y_train'], df_train['y_eval_proba'])

# Plotting ROC curve - training
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=fpr1, y=tpr1, mode='lines', name='ROC curve (area = %0.2f)' % roc_auc1,
                                   line=dict(color='darkorange', width=2)))
fig1.add_shape(type='line', line=dict(dash='dash'),
                        x0=0, x1=1, y0=0, y1=1)
fig1.update_layout(title='Receiver Operating Characteristic (ROC) - Training',
                            xaxis_title='False Positive Rate',
                            yaxis_title='True Positive Rate',
                            legend=dict(x=0.1, y=0.9),
                            autosize=False,
                            width=600,
                            height=600)
fig1.show()

# Compute ROC curve and ROC area - testing
fpr2, tpr2, thresholds2 = roc_curve(df_pred['y_test'], df_pred['y_pred_proba'])
roc_auc2 = roc_auc_score(df_pred['y_test'], df_pred['y_pred_proba'])

# Plotting ROC curve - testing
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=fpr2, y=tpr2, mode='lines', name='ROC curve (area = %0.2f)' % roc_auc2,
                                   line=dict(color='darkorange', width=2)))
fig2.add_shape(type='line', line=dict(dash='dash'),
                        x0=0, x1=1, y0=0, y1=1)
fig2.update_layout(title='Receiver Operating Characteristic (ROC) - Testing',
                            xaxis_title='False Positive Rate',
                            yaxis_title='True Positive Rate',
                            legend=dict(x=0.1, y=0.9),
                            autosize=False,
                            width=600,
                            height=600)
fig2.show()



# Informations complémentaiures sur les périodes d'entrainement et de test

# Confidence of Predictions
fig1 = go.Figure()
fig1.add_trace(go.Histogram(x=df_train['y_eval_proba'], name='Train Probabilities'))
fig1.add_trace(go.Histogram(x=df_pred['y_pred_proba'], name='Test Probabilities'))
fig1.update_layout(title='Confidence of Predictions',
                   xaxis_title='Predicted Probability',
                   yaxis_title='Count',
                   barmode='overlay')
fig1.update_traces(opacity=0.75)
fig1.show()

# Feature Importances
try:
    importances = classifier.feature_importances_
    sorted_indices = np.argsort(importances)[::-1]
    
    fig2 = go.Figure()
    fig2.add_trace(go.Bar(x=df.iloc[:,lfeat].columns[sorted_indices], y=importances[sorted_indices]))
    fig2.update_layout(title='Feature Importances',
                       xaxis_title='Features',
                       yaxis_title='Importance')
    fig2.show()
except:
    pass

# Overfitting (comparing AUC scores for training and testing sets)
auc_train = roc_auc_score(df_train['y_train'], df_train['y_eval_proba'])
auc_test = roc_auc_score(df_pred['y_test'], df_pred['y_pred_proba'])

fig3 = go.Figure()
fig3.add_trace(go.Bar(x=['Train', 'Test'], y=[auc_train, auc_test]))
fig3.update_layout(title='AUC Scores for Train and Test Sets',
                   xaxis_title='Dataset',
                   yaxis_title='AUC Score')
fig3.show()


